In [1]:
from roboflow import Roboflow
import open3d as o3d
import numpy as np
import cv2
from PIL import Image
import time
import colorsys
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
show_pcl = True

In [3]:
def visualizer(cloud):

    if type(cloud) != list:
        cloud = [cloud]

    if show_pcl:
        # o3d.visualization.draw_geometries(cloud,
        #                             zoom=0.57899999999999985,
        #                             front=  [ 0.053781796277127751, 0.99511863815317547, 0.082743062484869914 ],
        #                             up =  [ -0.99452345262671604, 0.045944145215534776, 0.093873519673282182 ],
        #                             lookat=[ 0.14950467828195535, -0.21448131248991498, 0.63221199653621662 ])
        
        center_of_mass = np.mean(np.asarray(cloud[0].points), axis=0)
        
        # print("com", center_of_mass)
        o3d.visualization.draw_geometries(cloud,
                                    zoom=0.7,
                                    front=[0, 1, 0],
                                    lookat=center_of_mass,
                                    up=[0, 0, 1])
    

In [4]:
def object_detection(img_path, model):

    prediction_data = model.predict(img_path, confidence=80, overlap=30).json()

    # visualize your prediction
    model.predict(img_path, confidence=80, overlap=30).save(img_path)
    return prediction_data

In [5]:
def get_selected_points(x_min, x_max, z_min, z_max, grid, data):
    selected_colors = []
    selected_points = []
    # img_test = cv2.imread('predictions\prediction22.jpg')

    for x in range(x_min, x_max, 1):
        for z in range(z_min, z_max, 1):
            idx = grid[x, z]
            if idx != -1:
                selected_colors.append(data[idx][0])
                selected_points.append(data[idx][1])
            # img_test[x, z] = [255, 0, 0] #BRG

    return np.array(selected_points), np.array(selected_colors)

In [6]:
def read_pcd(path):
    pcd = o3d.io.read_point_cloud(path)

    visualizer(pcd)

    return pcd

In [7]:
def get_bounding_box(prediction_data):

    # 2D bounding box coordinates
    bounding_box = {
        'z': prediction_data['predictions'][0]['x'],
        'x': prediction_data['predictions'][0]['y'],
        'width': prediction_data['predictions'][0]['width'],
        'height': prediction_data['predictions'][0]['height'],
    }

    # Image dimensions (to scale 2D to 3D)
    # image_width = prediction_data['image']['width']
    # image_height = prediction_data['image']['height']

    # Calculate 3D coordinates (assuming z-coordinate is arbitrary)
    x_min = int(bounding_box['x'] - bounding_box['height']/2)
    z_min = int(bounding_box['z'] - bounding_box['width']/2)
    x_max = int(bounding_box['x']  + bounding_box['height']/2)
    z_max = int(bounding_box['z'] + bounding_box['width']/2)

    return x_min, x_max, z_min, z_max

In [8]:
def draw_3d_bounding_box(pcd, all_pcds):

    for cld in all_pcds:

        bbox = cld.get_oriented_bounding_box()
        # both are the same
        # bbox = cld.get_axis_aligned_bounding_box()
        cld.paint_uniform_color((255, 0, 0))

        visualizer([cld, pcd, bbox])

        pcd = pcd.crop(bbox)

        visualizer(pcd)

    o3d.io.write_point_cloud(f"final_pcd.ply", pcd)

    # visualizer([pcd])

In [9]:
def algorithm(points, colors, shape1, shape2, resolution, reverse, axis_s, axes):
    check_grid = np.zeros((shape1, shape2)) #They will set to 1 if that [x, y] grid is used; 0 -> false
    grid = np.ones((shape1, shape2)) * -1 #the actual grid which will have indexes mapped to a map
    img = np.ones((shape1, shape2, 3), dtype=np.uint8) * 255
    data = {} #{-1:[('RGB'), ('XYZ')], } #the data. key is index number saved into grid and values are the RGB and xyz data

    points_sort = points[np.argsort(points[:, axis_s])[::reverse]]
    colors_sort = colors[np.argsort(points[:, axis_s])[::reverse]]

    max_val = np.max(np.asarray(points_sort), axis=0)
    min_val = np.min(np.asarray(points_sort), axis=0)

    first_a, snd_a = axes

    cent_x = (min_val[first_a]+max_val[first_a])/2
    range_x = np.abs(min_val[first_a]) + np.abs(max_val[first_a])
    x_r_new = shape1 - 1

    cent_z = (min_val[snd_a]+max_val[snd_a])/2
    range_z = np.abs(min_val[snd_a]) + np.abs(max_val[snd_a])
    z_r_new = shape1 - 1

    for idx, val in enumerate(points_sort):
        x_c = int(((val[first_a] - cent_x) / range_x) * x_r_new + x_r_new/2)
        z_c = int(((val[snd_a] - cent_z) / range_z) * z_r_new + z_r_new/2)

        if not check_grid[x_c, z_c]:

            check_grid[x_c, z_c] = 1
            data[idx] = [colors_sort[idx], val]
            grid[x_c, z_c] = idx
            col = np.uint8(colors_sort[idx]*255)
            img[x_c, z_c] = col
            # do around the point too

            for dx in range(-resolution, resolution, 1):
                for dz in range(-resolution, resolution, 1):

                    if 0 <= x_c + dx < shape1 and 0 <= z_c + dz < shape2:
                        if not check_grid[x_c + dx, z_c + dz]:
                            check_grid[x_c + dx, z_c + dz] = 1
                            img[x_c + dx, z_c + dz] = col
    
    return check_grid, grid, img, data

In [10]:
def save_img(image, img_path):
    image = Image.fromarray(image)
    image.save(img_path)
    print("Saved!")

In [44]:
def get_bounding_box_2d(img_path, rect_threshold):
    thresholded_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    edged = cv2.Canny(thresholded_image, 50, 100)

    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    bounding_boxes = []
    for contour in contours:
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)
        if (w*h) > rect_threshold:
            bounding_boxes.append((x, y, w, h))

    original_image = cv2.imread(img_path)  # Load your original image here
    bounding_box_image = original_image.copy()
    for (x, y, w, h) in bounding_boxes:
        cv2.rectangle(bounding_box_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw green rectangles

    # Display or save the image with bounding boxes
    cv2.imwrite(img_path, bounding_box_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # Optionally, you can print the bounding boxes
    # print("Bounding boxes:", bounding_boxes)

    return bounding_boxes #[[x, y, w, h]]

In [45]:
def color_detection(img_path):
    image = cv2.imread(img_path)

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_range = np.array([150, 160, 185])
    upper_range = np.array([360, 255, 255])

    mask = cv2.inRange(img_hsv, lower_range, upper_range)

    outside_range_mask = cv2.bitwise_not(mask)

    img_hsv[outside_range_mask != 0] = [0, 0, 255]

    result_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
    cv2.imwrite(img_path , result_image)

    bounding_boxes = get_bounding_box_2d(img_path, rect_threshold=2500)

    # return the maxes and mins of both the axes
    axes_ranges = []
    for x, y, w, h in bounding_boxes: #[x, y, w, h]
        x_min = int(x - h/2)
        z_min = int(y - w/2)
        x_max = int(x  + h/2)
        z_max = int(y + w/2)

        axes_ranges.append([x_min, x_max, z_min, z_max])

    return axes_ranges


In [31]:
x_shape = 3000
y_shape = 3000
z_shape = 3000

iterations = [
    {"shape": [y_shape, z_shape], "reverse": 1, "axis":0, "axes": [1, 2]}, #X
    {"shape": [x_shape, z_shape], "reverse": 1, "axis":1, "axes": [0, 2]}, #Y
    {"shape": [x_shape, y_shape], "reverse": 1, "axis":2, "axes": [0, 1]}, #Z
    {"shape": [y_shape, z_shape], "reverse": -1, "axis":0, "axes": [1, 2]}, #-X
    {"shape": [x_shape, z_shape], "reverse": -1, "axis":1, "axes": [0, 2]}, #-Y
    {"shape": [x_shape, y_shape], "reverse": -1, "axis":2, "axes": [0, 1]} #-Z
]

# pcd = o3d.io.read_point_cloud("../3dmodels/Bell_Pepper_6_26.ply")
pcd = o3d.io.read_point_cloud("../3dmodels/Strawberry_Flowers_nice.ply")

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd.get_center())

visualizer([pcd, mesh_frame])

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

rf = Roboflow(api_key="p0DKMSH5Ym3FH9zh7ZLa")
project = rf.workspace().project("sweet-pepper-detection")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [46]:
j = 0
color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [0, 255, 255], # Cyan
    [0, 128, 0],   # Green (dark)
]

In [32]:
threshold = 0.75 # 0 -> 1

x, y, z = np.asarray(pcd.points)[:, :3].T

std = np.std((x, y, z), axis=1) * threshold

hist_x, bins_x = np.histogram(x, bins=50)
hist_y, bins_y = np.histogram(y, bins=50)

max_freq_bin_x = bins_x[np.argmax(hist_x)]
max_freq_bin_y = bins_y[np.argmax(hist_y)]

min_bound = (max_freq_bin_x, max_freq_bin_y, -2) - std
max_bound = (max_freq_bin_x, max_freq_bin_y, 2) + std

bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                    max_bound=max_bound)
pcd = pcd.crop(bounding_box)

visualizer(pcd)

In [33]:
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

In [ ]:
all_point_clouds = []
all_points = []
all_bounding_vals = []
all_grid = []
all_data1 = []

for i in iterations:

    print("In iter", j)
    img_path=f"numpy/img_{j}.jpg"
    
    check_grid, grid, img, data = algorithm(
        points=points,
        colors=colors,
        shape1=i["shape"][0],
        shape2=i["shape"][1],
        resolution=11,
        reverse=i["reverse"],
        axis_s=i["axis"],
        axes=i["axes"])
    
    save_img(image=img, img_path=img_path)
    print("image saved")

    method = "color" # or object

    if method == "object":
        predictions = object_detection(img_path=img_path, model=model)

        if len(predictions['predictions']) == 0:
            print("No objects found")
            j+=1
            continue

        x_min, x_max, z_min, z_max = get_bounding_box(prediction_data=predictions)

    elif method == "color":
        x_min, x_max, z_min, z_max = color_detection(img_path=img_path)[0]
    
    selected_points, selected_colors = get_selected_points(x_min, x_max, z_min, z_max, grid, data)
    all_points.append(selected_points)

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(selected_points)
    point_cloud.paint_uniform_color(color[j])

    all_point_clouds.append(point_cloud)

    visualizer([point_cloud])

    merged = point_cloud+pcd

    o3d.io.write_point_cloud(f"pcd_{j}.ply", point_cloud)

    print("Done ", j)

    j += 1

print("Outside the loop")
visualizer(all_point_clouds)

# with open('output.txt', 'w') as f:
#     for sublist in all_points:
#         f.write(' '.join(map(str, sublist)) + '\n')

draw_3d_bounding_box(pcd, all_point_clouds)

In [ ]:
paths = ['pcd_0.ply', 'pcd_1.ply', 'pcd_3.ply', 'pcd_4.ply', 'pcd_5.ply']

all_pcd = []

for path in paths:
    temp_pcd = read_pcd(path)

    all_pcd.append(temp_pcd)

In [ ]:
def draw_3d_bounding_box_new(pcd, all_pcds):
    new_all = all_pcd.append(pcd)
    visualizer(all_pcd)

    for cld in all_pcds:

        bbox = cld.get_oriented_bounding_box()
        # both are the same
        # bbox = cld.get_axis_aligned_bounding_box()
        cld.paint_uniform_color((255, 0, 0))

        visualizer([cld, pcd, bbox])

        pcd = pcd.crop(bbox)

        visualizer(pcd)

    o3d.io.write_point_cloud(f"final_pcd.ply", pcd)

    # visualizer([pcd])

draw_3d_bounding_box_new(pcd, all_pcd)